In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import pandas as pd

# Load ESM2 pooled embeddings
X = np.load("esm2_features.npy")  # (236607, 480)

df = pd.read_csv("3_levels_EC.tsv", sep="\t")
labels = df["EC number"].astype(str).values

le = LabelEncoder()
y = le.fit_transform(labels)

num_classes = len(le.classes_)
print("Classes:", num_classes)


c:\Users\USER\Downloads\AI FINAL PROJECT TESTING\.venv311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classes: 263


In [2]:
class CNNDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).unsqueeze(1)  # (N, 1, 480)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None
)

train_ds = CNNDataset(X_train, y_train)
test_ds = CNNDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False)


In [3]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.BatchNorm1d(32),

            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.BatchNorm1d(64),

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            nn.AdaptiveAvgPool1d(1)  # → (batch, 128, 1)
        )

        self.fc = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv(x)
        x = x.squeeze(-1)  # remove last dim
        return self.fc(x)


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CNNClassifier(num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 20

for epoch in range(1, EPOCHS+1):
    model.train()
    total = 0
    correct = 0
    total_loss = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")

    for Xb, yb in pbar:
        Xb, yb = Xb.to(device), yb.to(device)

        optimizer.zero_grad()
        logits = model(Xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(yb)
        preds = logits.argmax(1)
        correct += (preds == yb).sum().item()
        total += len(yb)

        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch}: loss={total_loss/total:.4f}, acc={correct/total:.4f}")


Epoch 1/20: 100%|██████████| 740/740 [00:07<00:00, 93.79it/s, loss=2.45] 


Epoch 1: loss=3.5105, acc=0.2533


Epoch 2/20: 100%|██████████| 740/740 [00:06<00:00, 109.37it/s, loss=1.67]


Epoch 2: loss=2.0762, acc=0.5560


Epoch 3/20: 100%|██████████| 740/740 [00:06<00:00, 107.34it/s, loss=1.37]


...

Epoch 18: loss=0.3585, acc=0.9204


Epoch 19/20: 100%|██████████| 740/740 [00:06<00:00, 108.89it/s, loss=0.415]


Epoch 19: loss=0.3461, acc=0.9237


Epoch 20/20: 100%|██████████| 740/740 [00:06<00:00, 108.17it/s, loss=0.393]

Epoch 20: loss=0.3332, acc=0.9260


In [5]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for Xb, yb in tqdm(test_loader, desc="Evaluating"):
        Xb, yb = Xb.to(device), yb.to(device)
        logits = model(Xb)
        preds = logits.argmax(1)
        correct += (preds == yb).sum().item()
        total += len(yb)

print("Test accuracy:", correct / total)


Evaluating: 100%|██████████| 185/185 [00:00<00:00, 237.03it/s]

Test accuracy: 0.8819576518321288
